# ML.Net - Samples - TaxiFarePrediction

## Taxi Fare Prediction

| ML.NET version | API type          | Status                        | App Type    | Data type | Scenario            | ML Task                   | Algorithms                  |
|----------------|-------------------|-------------------------------|-------------|-----------|---------------------|---------------------------|-----------------------------|
| v1.5.2           | Dynamic API | Up-to-date | Jupyter Notebook | .csv files | Price prediction | Regression | Sdca Regression |

In this introductory sample, you'll see how to use [ML.NET](https://www.microsoft.com/net/learn/apps/machine-learning-and-ai/ml-dotnet) to predict taxi fares. In the world of machine learning, this type of prediction is known as **regression**.

In this sample, you'll see how to use [ML.NET](https://www.microsoft.com/net/learn/apps/machine-learning-and-ai/ml-dotnet) to predict whether a text message is spam. In the world of machine learning, this type of prediction is known as **binary classification**.

## Problem

This problem is centered around predicting the fare of a taxi trip in New York City. At first glance, it may seem to depend simply on the distance traveled. However, taxi vendors in New York charge varying amounts for other factors such as additional passengers, paying with a credit card instead of cash and so on. This prediction can be used in application for taxi providers to give users and drivers an estimate on ride fares.

To solve this problem, we will build an ML model that takes as inputs: 
* vendor ID
* rate code
* passenger count
* trip time
* trip distance
* payment type

and predicts the fare of the ride.

## ML task - Regression

The generalized problem of **regression** is to predict some continuous value for given parameters, for example:
* predict a house prise based on number of rooms, location, year built, etc.
* predict a car fuel consumption based on fuel type and car parameters.
* predict a time estimate for fixing an issue based on issue attributes.

The common feature for all those examples is that the parameter we want to predict can take any numeric value in certain range. In other words, this value is represented by `integer` or `float`/`double`, not by `enum` or `boolean` types.

## Solution

To solve this problem, first we will build an ML model. Then we will train the model on existing data, evaluate how good it is, and lastly we'll consume the model to predict taxi fares.

![Build -> Train -> Evaluate -> Consume](../shared_content/modelpipeline.png)

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 
#r "nuget:PLplot" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package PLplot version 5.13.7

Installed package Microsoft.ML version 1.5.2

## Using C# Class

In [2]:
using System;
using System.IO;
using System.IO.Compression;
using System.Linq;
using System.Net;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.Collections.Generic;
using System.Diagnostics;
using System.Globalization;
using static Microsoft.ML.TrainCatalogBase;
using static Microsoft.ML.DataOperationsCatalog;
using System.Diagnostics;
using PLplot;
using static Microsoft.ML.Transforms.NormalizingEstimator;


(13,7): warning CS0105: The using directive for 'System.Diagnostics' appeared previously in this namespace



## Declare data-classes for input data and predictions

In [3]:
public class TaxiTrip
{
    [LoadColumn(0)]
    public string VendorId;

    [LoadColumn(1)]
    public string RateCode;

    [LoadColumn(2)]
    public float PassengerCount;

    [LoadColumn(3)]
    public float TripTime;

    [LoadColumn(4)]
    public float TripDistance;

    [LoadColumn(5)]
    public string PaymentType;

    [LoadColumn(6)]
    public float FareAmount;
}

public class TaxiTripFarePrediction
{
    [ColumnName("Score")]
    public float FareAmount;
}


(19,19): warning CS0649: Field 'TaxiTrip.PaymentType' is never assigned to, and will always have its default value null

(22,18): warning CS0649: Field 'TaxiTrip.FareAmount' is never assigned to, and will always have its default value 0

(28,18): warning CS0649: Field 'TaxiTripFarePrediction.FareAmount' is never assigned to, and will always have its default value 0

(16,18): warning CS0649: Field 'TaxiTrip.TripDistance' is never assigned to, and will always have its default value 0

(13,18): warning CS0649: Field 'TaxiTrip.TripTime' is never assigned to, and will always have its default value 0

(7,19): warning CS0649: Field 'TaxiTrip.RateCode' is never assigned to, and will always have its default value null

(10,18): warning CS0649: Field 'TaxiTrip.PassengerCount' is never assigned to, and will always have its default value 0

(4,19): warning CS0649: Field 'TaxiTrip.VendorId' is never assigned to, and will always have its default value null



In [4]:
public class SingleTaxiTripSample
{
    internal static readonly TaxiTrip Trip1 = new TaxiTrip
    {
        VendorId = "VTS",
        RateCode = "1",
        PassengerCount = 1,
        TripDistance = 10.33f,
        PaymentType = "CSH",
        FareAmount = 0 // predict it. actual = 29.5
    };
}

### Constants

In [12]:
private static string TrainDataPath = @"./datasets/TaxiFarePrediction/taxi-fare-train.csv";
private static string TestDataPath = @"./datasets/TaxiFarePrediction/taxi-fare-test.csv";
private static string ModelPath = @"./datasets/TaxiFarePrediction/TaxiFareModel.zip";

private string args;
//args = "svg"; //or 
args = "";

### TaxiTripCsvReader

In [6]:
public class TaxiTripCsvReader
{
    public IEnumerable<TaxiTrip> GetDataFromCsv(string dataLocation, int numMaxRecords)
    {
        IEnumerable<TaxiTrip> records =
            File.ReadAllLines(dataLocation)
            .Skip(1)
            .Select(x => x.Split(','))
            .Select(x => new TaxiTrip()
            {
                VendorId = x[0],
                RateCode = x[1],
                PassengerCount = float.Parse(x[2], CultureInfo.InvariantCulture),
                TripTime = float.Parse(x[3], CultureInfo.InvariantCulture),
                TripDistance = float.Parse(x[4], CultureInfo.InvariantCulture),
                PaymentType = x[5],
                FareAmount = float.Parse(x[6], CultureInfo.InvariantCulture)
            })
            .Take<TaxiTrip>(numMaxRecords);

        return records;
    }
}

### ConsoleHelper

In [7]:
public static class ConsoleHelper
{
    public static void PrintPrediction(string prediction)
    {
        Console.WriteLine($"*************************************************");
        Console.WriteLine($"Predicted : {prediction}");
        Console.WriteLine($"*************************************************");
    }

    public static void PrintRegressionPredictionVersusObserved(string predictionCount, string observedCount)
    {
        Console.WriteLine($"-------------------------------------------------");
        Console.WriteLine($"Predicted : {predictionCount}");
        Console.WriteLine($"Actual:     {observedCount}");
        Console.WriteLine($"-------------------------------------------------");
    }

    public static void PrintRegressionMetrics(string name, RegressionMetrics metrics)
    {
        Console.WriteLine($"*************************************************");
        Console.WriteLine($"*       Metrics for {name} regression model      ");
        Console.WriteLine($"*------------------------------------------------");
        Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
        Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
        Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
        Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
        Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
        Console.WriteLine($"*************************************************");
    }

    public static void PrintBinaryClassificationMetrics(string name, CalibratedBinaryClassificationMetrics metrics)
    {
        Console.WriteLine($"************************************************************");
        Console.WriteLine($"*       Metrics for {name} binary classification model      ");
        Console.WriteLine($"*-----------------------------------------------------------");
        Console.WriteLine($"*       Accuracy: {metrics.Accuracy:P2}");
        Console.WriteLine($"*       Area Under Curve:      {metrics.AreaUnderRocCurve:P2}");
        Console.WriteLine($"*       Area under Precision recall Curve:  {metrics.AreaUnderPrecisionRecallCurve:P2}");
        Console.WriteLine($"*       F1Score:  {metrics.F1Score:P2}");
        Console.WriteLine($"*       LogLoss:  {metrics.LogLoss:#.##}");
        Console.WriteLine($"*       LogLossReduction:  {metrics.LogLossReduction:#.##}");
        Console.WriteLine($"*       PositivePrecision:  {metrics.PositivePrecision:#.##}");
        Console.WriteLine($"*       PositiveRecall:  {metrics.PositiveRecall:#.##}");
        Console.WriteLine($"*       NegativePrecision:  {metrics.NegativePrecision:#.##}");
        Console.WriteLine($"*       NegativeRecall:  {metrics.NegativeRecall:P2}");
        Console.WriteLine($"************************************************************");
    }

    public static void PrintMultiClassClassificationMetrics(string name, MulticlassClassificationMetrics metrics)
    {
        Console.WriteLine($"************************************************************");
        Console.WriteLine($"*    Metrics for {name} multi-class classification model   ");
        Console.WriteLine($"*-----------------------------------------------------------");
        Console.WriteLine($"    AccuracyMacro = {metrics.MacroAccuracy:0.####}, a value between 0 and 1, the closer to 1, the better");
        Console.WriteLine($"    AccuracyMicro = {metrics.MicroAccuracy:0.####}, a value between 0 and 1, the closer to 1, the better");
        Console.WriteLine($"    LogLoss = {metrics.LogLoss:0.####}, the closer to 0, the better");
        Console.WriteLine($"    LogLoss for class 1 = {metrics.PerClassLogLoss[0]:0.####}, the closer to 0, the better");
        Console.WriteLine($"    LogLoss for class 2 = {metrics.PerClassLogLoss[1]:0.####}, the closer to 0, the better");
        Console.WriteLine($"    LogLoss for class 3 = {metrics.PerClassLogLoss[2]:0.####}, the closer to 0, the better");
        Console.WriteLine($"************************************************************");
    }
    
    public static void PrintRegressionFoldsAverageMetrics(string algorithmName, IReadOnlyList<CrossValidationResult<RegressionMetrics>> crossValidationResults)
    {
        var L1 = crossValidationResults.Select(r => r.Metrics.MeanAbsoluteError);
        var L2 = crossValidationResults.Select(r => r.Metrics.MeanSquaredError);
        var RMS = crossValidationResults.Select(r => r.Metrics.RootMeanSquaredError);
        var lossFunction = crossValidationResults.Select(r => r.Metrics.LossFunction);
        var R2 = crossValidationResults.Select(r => r.Metrics.RSquared);

        Console.WriteLine($"*************************************************************************************************************");
        Console.WriteLine($"*       Metrics for {algorithmName} Regression model      ");
        Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
        Console.WriteLine($"*       Average L1 Loss:    {L1.Average():0.###} ");
        Console.WriteLine($"*       Average L2 Loss:    {L2.Average():0.###}  ");
        Console.WriteLine($"*       Average RMS:          {RMS.Average():0.###}  ");
        Console.WriteLine($"*       Average Loss Function: {lossFunction.Average():0.###}  ");
        Console.WriteLine($"*       Average R-squared: {R2.Average():0.###}  ");
        Console.WriteLine($"*************************************************************************************************************");
    }
    
    public static void PrintMulticlassClassificationFoldsAverageMetrics(
                                     string algorithmName,
                                   IReadOnlyList<CrossValidationResult<MulticlassClassificationMetrics>> crossValResults
                                                                       )
    {
        var metricsInMultipleFolds = crossValResults.Select(r => r.Metrics);

        var microAccuracyValues = metricsInMultipleFolds.Select(m => m.MicroAccuracy);
        var microAccuracyAverage = microAccuracyValues.Average();
        var microAccuraciesStdDeviation = CalculateStandardDeviation(microAccuracyValues);
        var microAccuraciesConfidenceInterval95 = CalculateConfidenceInterval95(microAccuracyValues);

        var macroAccuracyValues = metricsInMultipleFolds.Select(m => m.MacroAccuracy);
        var macroAccuracyAverage = macroAccuracyValues.Average();
        var macroAccuraciesStdDeviation = CalculateStandardDeviation(macroAccuracyValues);
        var macroAccuraciesConfidenceInterval95 = CalculateConfidenceInterval95(macroAccuracyValues);

        var logLossValues = metricsInMultipleFolds.Select(m => m.LogLoss);
        var logLossAverage = logLossValues.Average();
        var logLossStdDeviation = CalculateStandardDeviation(logLossValues);
        var logLossConfidenceInterval95 = CalculateConfidenceInterval95(logLossValues);

        var logLossReductionValues = metricsInMultipleFolds.Select(m => m.LogLossReduction);
        var logLossReductionAverage = logLossReductionValues.Average();
        var logLossReductionStdDeviation = CalculateStandardDeviation(logLossReductionValues);
        var logLossReductionConfidenceInterval95 = CalculateConfidenceInterval95(logLossReductionValues);

        Console.WriteLine($"*************************************************************************************************************");
        Console.WriteLine($"*       Metrics for {algorithmName} Multi-class Classification model      ");
        Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
        Console.WriteLine($"*       Average MicroAccuracy:    {microAccuracyAverage:0.###}  - Standard deviation: ({microAccuraciesStdDeviation:#.###})  - Confidence Interval 95%: ({microAccuraciesConfidenceInterval95:#.###})");
        Console.WriteLine($"*       Average MacroAccuracy:    {macroAccuracyAverage:0.###}  - Standard deviation: ({macroAccuraciesStdDeviation:#.###})  - Confidence Interval 95%: ({macroAccuraciesConfidenceInterval95:#.###})");
        Console.WriteLine($"*       Average LogLoss:          {logLossAverage:#.###}  - Standard deviation: ({logLossStdDeviation:#.###})  - Confidence Interval 95%: ({logLossConfidenceInterval95:#.###})");
        Console.WriteLine($"*       Average LogLossReduction: {logLossReductionAverage:#.###}  - Standard deviation: ({logLossReductionStdDeviation:#.###})  - Confidence Interval 95%: ({logLossReductionConfidenceInterval95:#.###})");
        Console.WriteLine($"*************************************************************************************************************");
    }    

    public static double CalculateStandardDeviation (IEnumerable<double> values)
    {
        double average = values.Average();
        double sumOfSquaresOfDifferences = values.Select(val => (val - average) * (val - average)).Sum();
        double standardDeviation = Math.Sqrt(sumOfSquaresOfDifferences / (values.Count()-1));
        return standardDeviation;
    }

    public static double CalculateConfidenceInterval95(IEnumerable<double> values)
    {
        double confidenceInterval95 = 1.96 * CalculateStandardDeviation(values) / Math.Sqrt((values.Count()-1));
        return confidenceInterval95;
    }

    public static void PrintClusteringMetrics(string name, ClusteringMetrics metrics)
    {
        Console.WriteLine($"*************************************************");
        Console.WriteLine($"*       Metrics for {name} clustering model      ");
        Console.WriteLine($"*------------------------------------------------");
        Console.WriteLine($"*       Average Distance: {metrics.AverageDistance}");
        Console.WriteLine($"*       Davies Bouldin Index is: {metrics.DaviesBouldinIndex}");
        Console.WriteLine($"*************************************************");
    }    
    
    public static void PeekDataViewInConsole(MLContext mlContext, IDataView dataView, IEstimator<ITransformer> pipeline, int numberOfRows = 4)
    {
        string msg = string.Format("Peek data in DataView: Showing {0} rows with the columns", numberOfRows.ToString());
        ConsoleWriteHeader(msg);

        //https://github.com/dotnet/machinelearning/blob/master/docs/code/MlNetCookBook.md#how-do-i-look-at-the-intermediate-data
        var transformer = pipeline.Fit(dataView);
        var transformedData = transformer.Transform(dataView);

        // 'transformedData' is a 'promise' of data, lazy-loading. call Preview  
        //and iterate through the returned collection from preview.

        var preViewTransformedData = transformedData.Preview(maxRows: numberOfRows);

        foreach (var row in preViewTransformedData.RowView)
        {
            var ColumnCollection = row.Values;
            string lineToPrint = "Row--> ";
            foreach (KeyValuePair<string, object> column in ColumnCollection)
            {
                lineToPrint += $"| {column.Key}:{column.Value}";
            }
            Console.WriteLine(lineToPrint + "\n");
        }
    }
    
    public static void PeekVectorColumnDataInConsole(MLContext mlContext, string columnName, IDataView dataView, IEstimator<ITransformer> pipeline, int numberOfRows = 4)
    {
        string msg = string.Format("Peek data in DataView: : Show {0} rows with just the '{1}' column", numberOfRows, columnName );
        ConsoleWriteHeader(msg);

        var transformer = pipeline.Fit(dataView);
        var transformedData = transformer.Transform(dataView);

        // Extract the 'Features' column.
        var someColumnData = transformedData.GetColumn<float[]>(columnName)
                                                    .Take(numberOfRows).ToList();

        // print to console the peeked rows

        int currentRow = 0;
        someColumnData.ForEach(row => {
                                        currentRow++;
                                        String concatColumn = String.Empty;
                                        foreach (float f in row)
                                        {
                                            concatColumn += f.ToString();                                              
                                        }

                                        Console.WriteLine();
                                        string rowMsg = string.Format("**** Row {0} with '{1}' field value ****", currentRow, columnName);
                                        Console.WriteLine(rowMsg);
                                        Console.WriteLine(concatColumn);
                                        Console.WriteLine();
                                      });
    }
    
    public static void ConsoleWriteHeader(params string[] lines)
    {
        var defaultColor = Console.ForegroundColor;
        Console.ForegroundColor = ConsoleColor.Yellow;
        Console.WriteLine(" ");
        foreach (var line in lines)
        {
            Console.WriteLine(line);
        }
        var maxLength = lines.Select(x => x.Length).Max();
        Console.WriteLine(new string('#', maxLength));
        Console.ForegroundColor = defaultColor;
    }

    public static void ConsoleWriterSection(params string[] lines)
    {
        var defaultColor = Console.ForegroundColor;
        Console.ForegroundColor = ConsoleColor.Blue;
        Console.WriteLine(" ");
        foreach (var line in lines)
        {
            Console.WriteLine(line);
        }
        var maxLength = lines.Select(x => x.Length).Max();
        Console.WriteLine(new string('-', maxLength));
        Console.ForegroundColor = defaultColor;
    }
    
}


warning CS1702: Assuming assembly reference 'System.Collections.Immutable, Version=1.2.3.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'Microsoft.ML.Data' matches identity 'System.Collections.Immutable, Version=1.2.5.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Collections.Immutable', you may need to supply runtime policy



### BuildTrainEvaluateAndSaveModel

In [13]:
private static ITransformer BuildTrainEvaluateAndSaveModel(MLContext mlContext)
{
    // STEP 1: Common data loading configuration
    IDataView baseTrainingDataView = mlContext.Data.LoadFromTextFile<TaxiTrip>(TrainDataPath, hasHeader: true, separatorChar: ',');
    IDataView testDataView = mlContext.Data.LoadFromTextFile<TaxiTrip>(TestDataPath, hasHeader: true, separatorChar: ',');

    //Sample code of removing extreme data like "outliers" for FareAmounts higher than $150 and lower than $1 which can be error-data 
    var cnt = baseTrainingDataView.GetColumn<float>(nameof(TaxiTrip.FareAmount)).Count();
    IDataView trainingDataView = mlContext.Data.FilterRowsByColumn(baseTrainingDataView, nameof(TaxiTrip.FareAmount), lowerBound: 1, upperBound: 150);
    var cnt2 = trainingDataView.GetColumn<float>(nameof(TaxiTrip.FareAmount)).Count();

    // STEP 2: Common data process configuration with pipeline data transformations
    var dataProcessPipeline = mlContext.Transforms.CopyColumns(outputColumnName: "Label", inputColumnName: nameof(TaxiTrip.FareAmount))
                    .Append(mlContext.Transforms.Categorical.OneHotEncoding(outputColumnName: "VendorIdEncoded", inputColumnName: nameof(TaxiTrip.VendorId)))
                    .Append(mlContext.Transforms.Categorical.OneHotEncoding(outputColumnName: "RateCodeEncoded", inputColumnName: nameof(TaxiTrip.RateCode)))
                    .Append(mlContext.Transforms.Categorical.OneHotEncoding(outputColumnName: "PaymentTypeEncoded",inputColumnName: nameof(TaxiTrip.PaymentType)))
                    .Append(mlContext.Transforms.NormalizeMeanVariance(outputColumnName: nameof(TaxiTrip.PassengerCount)))
                    .Append(mlContext.Transforms.NormalizeMeanVariance(outputColumnName: nameof(TaxiTrip.TripTime)))
                    .Append(mlContext.Transforms.NormalizeMeanVariance(outputColumnName: nameof(TaxiTrip.TripDistance)))
                    .Append(mlContext.Transforms.Concatenate("Features", "VendorIdEncoded", "RateCodeEncoded", "PaymentTypeEncoded", nameof(TaxiTrip.PassengerCount)
                    , nameof(TaxiTrip.TripTime), nameof(TaxiTrip.TripDistance)));

    // (OPTIONAL) Peek data (such as 5 records) in training DataView after applying the ProcessPipeline's transformations into "Features" 
    ConsoleHelper.PeekDataViewInConsole(mlContext, trainingDataView, dataProcessPipeline, 5);
    ConsoleHelper.PeekVectorColumnDataInConsole(mlContext, "Features", trainingDataView, dataProcessPipeline, 5);

    // STEP 3: Set the training algorithm, then create and config the modelBuilder - Selected Trainer (SDCA Regression algorithm)                            
    var trainer = mlContext.Regression.Trainers.Sdca(labelColumnName: "Label", featureColumnName: "Features");
    var trainingPipeline = dataProcessPipeline.Append(trainer);

    // STEP 4: Train the model fitting to the DataSet
    //The pipeline is trained on the dataset that has been loaded and transformed.
    Console.WriteLine("=============== Training the model ===============");
    var trainedModel = trainingPipeline.Fit(trainingDataView);

    // STEP 5: Evaluate the model and show accuracy stats
    Console.WriteLine("===== Evaluating Model's accuracy with Test data =====");

    IDataView predictions = trainedModel.Transform(testDataView);
    var metrics = mlContext.Regression.Evaluate(predictions, labelColumnName: "Label", scoreColumnName: "Score");

    ConsoleHelper.PrintRegressionMetrics(trainer.ToString(), metrics);

    // STEP 6: Save/persist the trained model to a .ZIP file
    mlContext.Model.Save(trainedModel, trainingDataView.Schema, ModelPath);

    Console.WriteLine("The model is saved to {0}", ModelPath);

    return trainedModel;
}


(13,31): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.9.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



### TestSinglePrediction

In [14]:
private static void TestSinglePrediction(MLContext mlContext)
{
    //Sample: 
    //vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
    //VTS,1,1,1140,3.75,CRD,15.5

    var taxiTripSample = new TaxiTrip()
    {
        VendorId = "VTS",
        RateCode = "1",
        PassengerCount = 1,
        TripTime = 1140,
        TripDistance = 3.75f,
        PaymentType = "CRD",
        FareAmount = 0 // To predict. Actual/Observed = 15.5
    };

    ///
    ITransformer trainedModel = mlContext.Model.Load(ModelPath, out var modelInputSchema);

    // Create prediction engine related to the loaded trained model
    var predEngine = mlContext.Model.CreatePredictionEngine<TaxiTrip, TaxiTripFarePrediction>(trainedModel);

    //Score
    var resultprediction = predEngine.Predict(taxiTripSample);
    ///

    Console.WriteLine($"**********************************************************************");
    Console.WriteLine($"Predicted fare: {resultprediction.FareAmount:0.####}, actual fare: 15.5");
    Console.WriteLine($"**********************************************************************");
}        

### PlotRegressionChart

In [15]:
private static void PlotRegressionChart(MLContext mlContext,                                               
                                        string testDataSetPath,
                                        int numberOfRecordsToRead,
                                        string args)
{
    ITransformer trainedModel;
    using (var stream = new FileStream(ModelPath, FileMode.Open, FileAccess.Read, FileShare.Read))
    {
        trainedModel = mlContext.Model.Load(stream, out var modelInputSchema);
    }

    // Create prediction engine related to the loaded trained model
    var predFunction = mlContext.Model.CreatePredictionEngine<TaxiTrip, TaxiTripFarePrediction>(trainedModel);

    string chartFileName = "";
    using (var pl = new PLStream())
    {
        // use SVG backend and write to SineWaves.svg in current directory
        if (args.Length == 1 && args == "svg")
        {
            pl.sdev("svg");
            chartFileName = "TaxiRegressionDistribution.svg";
            pl.sfnam(chartFileName);
        }
        else
        {
            pl.sdev("pngcairo");
            chartFileName = "TaxiRegressionDistribution.png";
            pl.sfnam(chartFileName);
        }

        // use white background with black foreground
        pl.spal0("cmap0_alternate.pal");

        // Initialize plplot
        pl.init();

        // set axis limits
        const int xMinLimit = 0;
        const int xMaxLimit = 35; //Rides larger than $35 are not shown in the chart
        const int yMinLimit = 0;
        const int yMaxLimit = 35;  //Rides larger than $35 are not shown in the chart
        pl.env(xMinLimit, xMaxLimit, yMinLimit, yMaxLimit, AxesScale.Independent, AxisBox.BoxTicksLabelsAxes);

        // Set scaling for mail title text 125% size of default
        pl.schr(0, 1.25);

        // The main title
        pl.lab("Measured", "Predicted", "Distribution of Taxi Fare Prediction");

        // plot using different colors
        // see http://plplot.sourceforge.net/examples.php?demo=02 for palette indices
        pl.col0(1);

        int totalNumber = numberOfRecordsToRead;
        var testData = new TaxiTripCsvReader().GetDataFromCsv(testDataSetPath, totalNumber).ToList();

        //This code is the symbol to paint
        char code = (char)9;

        // plot using other color
        //pl.col0(9); //Light Green
        //pl.col0(4); //Red
        pl.col0(2); //Blue

        double yTotal = 0;
        double xTotal = 0;
        double xyMultiTotal = 0;
        double xSquareTotal = 0;

        for (int i = 0; i < testData.Count; i++)
        {
            var x = new double[1];
            var y = new double[1];

            //Make Prediction
            var FarePrediction = predFunction.Predict(testData[i]);

            x[0] = testData[i].FareAmount;
            y[0] = FarePrediction.FareAmount;

         //Paint a dot
         pl.poin(x, y, code);

         xTotal += x[0];
         yTotal += y[0];

         double multi = x[0] * y[0];
         xyMultiTotal += multi;

         double xSquare = x[0] * x[0];
         xSquareTotal += xSquare;

         double ySquare = y[0] * y[0];

         Console.WriteLine($"-------------------------------------------------");
         Console.WriteLine($"Predicted : {FarePrediction.FareAmount}");
         Console.WriteLine($"Actual:    {testData[i].FareAmount}");
         Console.WriteLine($"-------------------------------------------------");
     }

     // Regression Line calculation explanation:
     // https://www.khanacademy.org/math/statistics-probability/describing-relationships-quantitative-data/more-on-regression/v/regression-line-example

     double minY = yTotal / totalNumber;
     double minX = xTotal / totalNumber;
     double minXY = xyMultiTotal / totalNumber;
     double minXsquare = xSquareTotal / totalNumber;

     double m = ((minX * minY) - minXY) / ((minX * minX) - minXsquare);

     double b = minY - (m * minX);

     //Generic function for Y for the regression line
     // y = (m * x) + b;

     double x1 = 1;
     //Function for Y1 in the line
     double y1 = (m * x1) + b;

     double x2 = 39;
     //Function for Y2 in the line
     double y2 = (m * x2) + b;

     var xArray = new double[2];
     var yArray = new double[2];
     xArray[0] = x1;
     yArray[0] = y1;
     xArray[1] = x2;
     yArray[1] = y2;

     pl.col0(4);
     pl.line(xArray, yArray);

     // end page (writes output to disk)
     pl.eop();

     // output version of PLplot
     pl.gver(out var verText);
     Console.WriteLine("PLplot version " + verText);

 } // the pl object is disposed here

 // Open Chart File In Microsoft Photos App (Or default app, like browser for .svg)

 Console.WriteLine("Showing chart...");
 var p = new Process();
 string chartFileNamePath = @".\" + chartFileName;
 p.StartInfo = new ProcessStartInfo(chartFileNamePath)
 {
     UseShellExecute = true
 };
 p.Start();
}

## Evaluate

In [17]:
//Create ML Context with seed for repeatable/deterministic results
MLContext mlContext = new MLContext(seed: 0);

// Create, Train, Evaluate and Save a model
BuildTrainEvaluateAndSaveModel(mlContext);

// Make a single test prediction loding the model from .ZIP file
TestSinglePrediction(mlContext);

// Paint regression distribution chart for a number of elements read from a Test DataSet file
PlotRegressionChart(mlContext, TestDataPath, 100, args);

Console.WriteLine("====== The End ======");


 
Peek data in DataView: Showing 5 rows with the columns
######################################################
Row--> | VendorId:CMT| RateCode:1| PassengerCount:1| PassengerCount:0.42661247| TripTime:1271| TripTime:1.5157564| TripDistance:3.8| TripDistance:0.8772732| PaymentType:CRD| FareAmount:17.5| Label:17.5| VendorIdEncoded:1| VendorIdEncoded:Sparse vector of size 2, 1 explicit values| RateCodeEncoded:1| RateCodeEncoded:Sparse vector of size 6, 1 explicit values| PaymentTypeEncoded:1| PaymentTypeEncoded:Sparse vector of size 5, 1 explicit values| Features:Sparse vector of size 16, 6 explicit values

Row--> | VendorId:CMT| RateCode:1| PassengerCount:1| PassengerCount:0.42661247| TripTime:474| TripTime:0.5652782| TripDistance:1.5| TripDistance:0.34629205| PaymentType:CRD| FareAmount:8| Label:8| VendorIdEncoded:1| VendorIdEncoded:Sparse vector of size 2, 1 explicit values| RateCodeEncoded:1| RateCodeEncoded:Sparse vector of size 6, 1 explicit values| PaymentTypeEncoded:1| PaymentType

Unhandled exception: System.DllNotFoundException: Unable to load shared library '/home/dramos/.nuget/packages/plplot/5.13.7/runtimes/linux-x64/native/libplplot.so' or one of its dependencies. In order to help diagnose loading problems, consider setting the LD_DEBUG environment variable: libplplot.so.15: cannot open shared object file: No such file or directory
   at System.Runtime.Loader.AssemblyLoadContext.InternalLoadUnmanagedDllFromPath(String unmanagedDllPath)
   at System.Runtime.Loader.AssemblyLoadContext.LoadUnmanagedDllFromPath(String unmanagedDllPath)
   at Microsoft.DotNet.DependencyManager.NativeAssemblyLoadContext.LoadNativeLibrary(String path) in F:\workspace\_work\1\s\src\fsharp\Microsoft.DotNet.DependencyManager\NativeDllResolveHandler.fs:line 46
   at Microsoft.DotNet.DependencyManager.NativeDllResolveHandlerCoreClr._resolveUnmanagedDll(Assembly _arg1, String name) in F:\workspace\_work\1\s\src\fsharp\Microsoft.DotNet.DependencyManager\NativeDllResolveHandler.fs:line 11